In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
#daily cases covid tested positive...
#daily death rate
covid=pd.read_csv('/kaggle/input/us-counties-covid19-weather-sociohealth-data/US_counties_COVID19_health_weather_data.csv')


In [ ]:
covid

In [ ]:
covidgb=covid.groupby(['state','date']).mean()

In [ ]:
covidgb2=covid.groupby(['state','date']).sum()
covidgb2

In [ ]:
lijst=['cases','deaths','area_sqmi','num_deaths']
for li in lijst:
    covidgb[li]=covidgb2[li]/covidgb2['total_population']
covidgb['total_population']=covidgb2['total_population']

In [ ]:
col=covidgb.describe().T.index
col

In [ ]:
def clustertechniquestf(dtrain,termtf,label,indexv):
    #print(dtrain)    
    ydata=dtrain.iloc[:,7:]
    !pip install dabl
    import dabl
    dtrain = dabl.clean(dtrain, verbose=1)
    dabl.plot(dtrain, label)
    if False:
        from sklearn.decomposition import TruncatedSVD
        tsvd=TruncatedSVD(n_components=50, n_iter=7, random_state=42)
        from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

        print('#vectorizing text')
        vect=TfidfVectorizer()
        dfidf=vect.fit_transform(dtrain[termtf]+' '+dtrain['Genres']+ dtrain['Category']) 
        dfidf=tsvd.fit_transform(dfidf)
        for xi in range(50):
            dtrain['svd'+str(xi)]=dfidf[:,xi]
        dtrain=dtrain.drop(termtf,axis=1)
    if False:
        kolom=dtrain.describe().T
        from sklearn.preprocessing import OneHotEncoder
        toencode=[ci for ci in dtrain.columns if ci not in list(kolom.index)+[indexv,label]]
        print('to encode',toencode)
        ohe=OneHotEncoder()
        data2=ohe.fit_transform(dtrain[toencode].fillna('')).toarray()
        dtrain=dtrain.drop(toencode,axis=1)
        #from sklearn.preprocessing import OneHotEncoder
        #data2=OneHotEncoder().fit_transform(data[['Gender']]).toarray()
        tel=0
        for ci in ohe.get_feature_names():
            dtrain[ci]=data2[:,tel]
            tel+=1
        dtrain[label]=ydata[label]
    #add random columns
    if False:
        for xi in range(10):
            labnm='rand'+str(xi)
            dtrain[labnm]=np.random.randint(0,10,size=(len(dtrain), 1))
    
    print('#encodings',dtrain.shape)
    cols=[ci for ci in dtrain.columns if ci not in [indexv,'index',label]]
    dtest=dtrain[dtrain[label].isnull()==True][[indexv,label]]
    print(dtest)

    print('encodings  after shape',dtrain.shape)
    from sklearn.metrics.pairwise import cosine_similarity,laplacian_kernel,manhattan_distances
    
   ########### cosine similarity check
    if False:
        ydata=dtrain[[indexv,label]]        
        dtrain['grb']=np.round(dtrain[label]/dtrain[label].max())
        print(dtrain)
        datagb=dtrain.groupby(['grb']).median()
        print(datagb)
        dtrain=datagb.dropna().append(dtrain)
        ydata=dtrain[[indexv,label]]
        print(ydata)
        data2=pd.DataFrame(cosine_similarity(dtrain.drop(['grb',label],axis=1),datagb.drop(label,axis=1) ))
        tel=0
        for ci in ohe.get_feature_names():
            dtrain[ci]=data2[:,tel]
            tel+=1
        
        dtrain[label]=ydata[label]
        dtrain[indexv]=ydata[indexv]
        ydata=dtrain[[indexv,label]]    
    
    #split data or use splitted data
    Xtrain=dtrain[dtrain[label].isnull()==False].drop([indexv,label],axis=1).fillna(0)
    Y_train=dtrain[dtrain[label].isnull()==False][label]
    Xtest=dtrain[dtrain[label].isnull()==True].drop([indexv,label],axis=1).fillna(0)
    Y_test=dtrain[dtrain[label].isnull()==True][label].fillna(0)
    print(Y_test)
    for xi in range(len(Y_test)):
        Y_test.iloc[xi]=np.random.random((1,1))[0]
    print(Y_test)
    if len(Xtest)==0:
        from sklearn.model_selection import train_test_split
        Xtrain,Xtest,Y_train,Y_test = train_test_split(dtrain.drop(label,axis=1).fillna(0),dtrain[label],test_size=0.25,random_state=0)
    lenxtr=len(Xtrain)

    print('splitting data train test X-y',Xtrain.shape,Y_train.shape,Xtest.shape,Y_test.shape)
   
    n_folds = 5
    def rmsle_cv(model):
        kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_total_clu[:lenxtr])
        rmse= np.sqrt(-cross_val_score(model, X_total_clu[:lenxtr], Y_train, scoring="neg_mean_squared_error", cv = kf))
        return(rmse)

    import matplotlib.pyplot as plt 
    from sklearn.preprocessing import QuantileTransformer,RobustScaler,Normalizer,MaxAbsScaler,MinMaxScaler
    
    from sklearn.decomposition import PCA,TruncatedSVD,NMF,FastICA
    from umap import UMAP  # knn lookalike of tSNE but faster, so scales up
    from sklearn.manifold import TSNE #limit number of records to 100000
    import xgboost as xgb
    scalers =[Dummy(1),
              #MinMaxScaler(),
             #RobustScaler(),
             #MaxAbsScaler(),
             #Normalizer(),
             #QuantileTransformer(output_distribution='uniform')
    ]
    clusters = [Dummy(1),
               PCA(n_components=20,random_state=0,whiten=True),
               #FastICA(n_components=15,random_state=0),
               #TruncatedSVD(n_components=15, n_iter=7, random_state=42),
                #NMF(n_components=10,random_state=0),            
                #UMAP(n_neighbors=5,n_components=3, min_dist=0.3,metric='minkowski'),
                #TSNE(n_components=2,random_state=0)
                ] 
    clunaam=['raw','PCA','ica','tSVD','UMAP','tSNE']#,'ICA','tSVD','nmf','UMAP','tSNE']
    
    from lightgbm import LGBMClassifier,LGBMRegressor
    
    from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
    from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
    from sklearn.model_selection import KFold, cross_val_score, train_test_split

    from sklearn.svm import SVC, LinearSVC,NuSVC
    from sklearn.multiclass import OneVsRestClassifier
    from sklearn.ensemble import RandomForestRegressor,ExtraTreesRegressor, RandomForestClassifier,ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier
    from sklearn.neural_network import MLPClassifier,MLPRegressor
    from sklearn.linear_model import PassiveAggressiveClassifier,Perceptron,SGDClassifier,LogisticRegression,Lasso
    import xgboost as xgb
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
    from sklearn.linear_model import ElasticNetCV,ridge_regression,HuberRegressor,LinearRegression,BayesianRidge,RANSACRegressor
    from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
    from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error,mean_squared_log_error
    from sklearn.ensemble import BaggingRegressor,VotingRegressor

    classifiers = [LGBMRegressor(),
                   xgb.XGBRegressor(n_estimators=100, reg_lambda=1,gamma=0,  max_depth=3),
                   #GradientBoostingRegressor(),
                   ExtraTreesRegressor(n_jobs=4),
                   RandomForestRegressor(random_state=1, n_estimators=10),      
                   ElasticNet(),
                   #VotingRegressor([('lrg', LGBMRegressor()), ('xg', xgb.XGBRegressor()),('xt',ExtraTreesRegressor(n_jobs=4)),('kn',RandomForestRegressor(random_state=1, n_estimators=10))],n_jobs=4),
                   #BaggingRegressor(base_estimator=Lasso(alpha =0.0005, random_state=1)),
                   BayesianRidge(),
                   # Lasso(alpha =0.0005, random_state=1),
                   #RANSACRegressor(),
                   KNeighborsRegressor(),
                   #ElasticNetCV(cv=5, random_state=0),
                   #HuberRegressor(),
                   #LinearRegression(),
                  ]
    clanaam= ['lgbm','xgb','Gboost','Xtree','rFor','vote','bagg','lasso','BaysR','Ransac','KNNr','elast','huber','linear',]
    from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
    
    results=[]

    for sca in scalers:
        X_train = sca.fit_transform(Xtrain)
        X_test = sca.transform(Xtest)
            
        #cluster data
        for clu in clusters:
            clunm=clunaam[clusters.index(clu)] #find naam
            clunm=str(clu)[:10]
            X_total_clu = clu.fit_transform(np.concatenate( (X_train,X_test),axis=0))
            if False:
                X_total_clu=np.concatenate((X_total_clu,np.concatenate( (X_train,X_test),axis=0)),axis=1)
            print(X_total_clu.shape)
            plt.scatter(X_total_clu[:lenxtr,0],X_total_clu[:lenxtr,1],c=Y_train.values,cmap='prism')
            plt.title(clu)
            plt.show()

            #classifiy 
            for cla in classifiers:
                import datetime
                start = datetime.datetime.now()
                #clanm=clanaam[classifiers.index(cla)] #find naam
                clanm=str(cla)[:10]
                print('    ',cla)
                print(rmsle_cv(cla).mean())
                #cla.fit(X_total_clu,np.concatenate( (Y_train,Y_test)) )
                cla.fit(X_total_clu[:lenxtr],Y_train )

                #predict
                trainpredi=cla.predict(X_total_clu[:lenxtr])

                #print(classification_report(trainpredi,Y_train))            
                testpredi=cla.predict(X_total_clu[lenxtr:])  
                try:
                    trainprediprob=cla.predict(X_total_clu[:lenxtr])
                    testprediprob=cla.predict(X_total_clu[lenxtr:]) 
                    plt.scatter(x=trainprediprob,y=Y_train,marker="x",alpha=0.53)
                    plt.show()
                    plt.scatter(x=testprediprob, y=testpredi, marker='.', alpha=0.53)
                    plt.show()
                except:
                    print()
                #testpredi=converging(pd.DataFrame(X_train),pd.DataFrame(X_test),Y_train,pd.DataFrame(testpredi),Y_test,clu,cla) #PCA(n_components=10,random_state=0,whiten=True),MLPClassifier(alpha=0.510,activation='logistic'))
                try:
                    feat=pd.DataFrame(cla.feature_importances_)
                    feat['fields']=Xtrain.columns
                    print(feat.sort_values(0)[-10:])
                except:
                    print()
                if len(dtest)==0:
                    test_score=cla.score(X_total_clu[lenxtr:],Y_test)
                    mse = mean_squared_error(testpredi,Y_test)
                    train_score=cla.score(X_total_clu[:lenxtr],Y_train)

                    
                    
                    r2s=r2_score(testpredi,Y_test)  
                    mse=mean_squared_error(testpredi,Y_test)  
                    mae=mean_absolute_error(testpredi,Y_test)
                    li = [clunm,clanm,train_score,test_score,mse,mae,r2s,np.sqrt(mse)]
                    results.append(li)
                    print('Test r2score',r2s,'mse',mse,'rmse',np.sqrt(mse),'mae',mae,pd.DataFrame(results) )

                    plt.title(clanm+'test corr & mse:'+np.str(test_score)+' '+np.str(mse)+' and test confusionmatrix')
                    plt.scatter(x=Y_test, y=testpredi, marker='.', alpha=1)
                    plt.scatter(x=[np.mean(Y_test)], y=[np.mean(testpredi)], marker='o', color='red')
                    plt.xlabel('Real test'); plt.ylabel('Pred. test')
                    plt.show()


                else:
    #                testpredlabel=le.inverse_transform(testpredi)  #use if you labellezid the classes 



                    testpredlabel=testpredi
                    try:
                        rmsle=np.sqrt(mean_squared_log_error( trainpredi, Y_train ))
                    except:
                        rmsle='xxx'
                    print('train correl',r2_score(trainpredi,Y_train),'mse ',mean_squared_error(trainpredi,Y_train),'rmse',np.sqrt(mean_squared_error(trainpredi,Y_train)),'mae',mean_absolute_error(trainpredi,Y_train),'rmsle',rmsle)
                    results.append([str(sca)[:10],clunm,clanm,r2_score(trainpredi,Y_train),mean_squared_error(trainpredi,Y_train),np.sqrt(mean_squared_error(trainpredi,Y_train)),mean_absolute_error(trainpredi,Y_train),rmsle])
                    print(pd.DataFrame(results).sort_values(6))
                    submit = pd.DataFrame({'Id': dtest[indexv].astype('int'),'PredictedValue': testpredlabel})
                    #submit['id']=submit['id'].astype('int')

                    filenaam='subm_'+clunm+'_'+clanm+'_'+str(sca)[:10]+'.csv'
                    submit.to_csv(path_or_buf =filenaam, index=False)
                    print(submit.head())

                print(clanm,clunm,sca,'0 classifier time',datetime.datetime.now()-start)
            
    if len(dtest)==0:       
        print(pd.DataFrame(results).sort_values(3))
        submit=[]
    return submit

#Custom Transformer that extracts columns passed as argument to its constructor 
class Dummy( ):
    #Class Constructor 
    def __init__( self, feature_names ):
        self._feature_names = feature_names 
    
    #Return self nothing else to do here    
    def fit( self, X, y = None ):
        return self 
    
    #Method that describes what we need this transformer to do
    def fit_transform(self, X, y = None ):
        return X
    
    def transform(self, X, y = None ):
        return X

#total.columns=[0,1,2,3,4,5,6,7,'strength','index']

clustertechniquestf(covidgb2.drop(['cases'],axis=1),"","deaths",'area_sqmi') 
